In [201]:
# se utiliza para el manejo de rutas y directorios.
import os

# Calculo cientifico y vectorial para python
import numpy as np

# Librerias para graficar
from matplotlib import pyplot

# Modulo de optimización de scipy
from scipy import optimize

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [202]:
data = np.loadtxt('/content/ecoli1.txt', delimiter=',',dtype=str)
#reeemplazar los valores de la columna 8 guardar en y
#'cp' igual a 0     #'imU' igual a 4
#'im' igual a 1     # 'om' igual a 5
#'imS' igual a 2    # 'omL' igual 6
#'imL' igual a 3    #'pp' igual a 7

data2 = np.where(data[:,0:8]=='cp', 0,data[:,0:8])
data2 = np.where(data2[:,0:8]=='im', 1,data2[:,0:8])
data2 = np.where(data2[:,0:8]=='imS', 2,data2[:,0:8])
data2 = np.where(data2[:,0:8]=='imL', 3,data2[:,0:8])
data2 = np.where(data2[:,0:8]=='imU', 4,data2[:,0:8])
data2 = np.where(data2[:,0:8]=='om', 5,data2[:,0:8])
data2 = np.where(data2[:,0:8]=='omL', 6,data2[:,0:8])
data2 = np.where(data2[:,0:8]=='pp', 7,data2[:,0:8])
y = data2[:,7]
X = data2[:,0:7]


X = np.asarray(X, dtype=np.float64)
y = np.asarray(y, dtype=np.float64)


In [203]:
y

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
       2., 3., 3., 4., 4.

In [204]:
#separar los datos en variables de entrenamiento 70% de los datos  y 30% de prueba
#X = data2[0:int(0.7*len(data2)),0:8]
#y = y[0:int(0.7*len(y))]

#print(X.shape)
#print(y.shape)

#X_test = data2[int(0.7*len(data2)):len(data2),0:8]
#y_test = y[int(0.7*len(y)):len(y)]
#print(X_test.shape)
#print(y_test.shape)


In [205]:
input_layer_size  = 8
num_labels = 8
m = y.size

In [206]:
def calcularSigmoide(z):
    # Calcula la sigmoide de z.

    return 1.0 / (1.0 + np.exp(-z))

In [207]:
def calcularCosto(theta, X, y, lambda_):
    # Inicializa algunos valores utiles
    m = y.size
    
    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)
    
    J = 0
    grad = np.zeros(theta.shape)
    
    h = calcularSigmoide(X.dot(theta.T))
    
    temp = theta
    temp[0] = 0
    
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
    
    grad = (1 / m) * (h - y).dot(X)
    # Se aplica regularizacion en la siguiente linea 
    grad = grad + (lambda_ / m) * temp

    return J, grad

In [208]:
def oneVsAll(X, y, num_labels, lambda_):

    m, n = X.shape
    
    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)


    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 100}
        res = optimize.minimize(calcularCosto, 
                                initial_theta, 
                                (X, (y == (c + 1)), lambda_), 
                                jac=True, 
                                method='BFGS',
                                options=options) 
        
        all_theta[c] = res.x

    return all_theta

In [209]:
lambda_ = 0.001
all_theta = oneVsAll(X, y, num_labels, lambda_)

In [210]:
print(all_theta)

[[ -5.84719773  -8.71233262   2.99000133  -3.27582878  -2.9179322
    0.81491995  15.94153924   1.03036849]
 [ -5.06487759  10.3884157   -3.8457364   -6.10273037  -1.73582289
   -0.58014921   1.98250589  -3.1100017 ]
 [-26.81820815  11.16050055 -14.56977366  15.96053079  14.7492379
  -12.15721512   4.31449766   6.71896235]
 [ -7.46704155  11.36401658  -5.71143228  -0.19726896  -3.81024892
   -0.05662376  -1.06109498   6.26383295]
 [-22.77452555   6.84266985  14.01754013  -3.0363099  -10.67947179
   33.90628247  -6.49100184  -8.47198338]
 [ -7.20073558   7.95727126  -8.74911254  19.90019159 -13.67099095
   -0.27266247   3.17447731 -17.12640096]
 [  0.1528626    7.44305758  10.60733966 -11.16357436  -1.21723943
   -8.3968782   -1.98401869  -2.63520766]
 [ -4.56308274  -2.15913879  -2.21876533  -2.2431171   -2.28606784
   -2.22977655  -2.11535718  -2.15468554]]


In [211]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(calcularSigmoide(X.dot(all_theta.T)), axis = 1)

    return p + 1

In [212]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y_train) * 100))

XPrueba =np.asarray([[0.25,0.48,0.48,0.50,0.44,0.17,0.29]])
m,n = XPrueba.shape
XPrueba = np.concatenate((np.ones((m, 1)), XPrueba), axis=1)
#print(XPrueba)
# print(XPrueba.shape)
# print(XPrueba)
p = np.argmax(calcularSigmoide(XPrueba.dot(all_theta.T)), axis = 1)
print(p + 1)

(336, 7)
Precision del conjuto de entrenamiento: 0.00%
[7]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
